In [32]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
import tensorflow
# tensorflow.keras.__version__

In [33]:
# Read in csv (already transformed)
hosp_df = pd.read_csv('..//..//data/hospitals/All_Hospitals.csv')

# Drop columns not need for modeling
df = hosp_df.drop(['Description', 'FAC_NO', 'FAC_NAME','FAC_STR_ADDR','FAC_CITY','FAC_PHONE','FAC_ADMIN_NAME','FAC_OPERATED_THIS_YR','FAC_OP_PER_BEGIN_DT','FAC_OP_PER_END_DT','FAC_PAR_CORP_NAME','FAC_PAR_CORP_BUS_ADDR','FAC_PAR_CORP_CITY','FAC_PAR_CORP_STATE','REPT_PREP_NAME','SUBMITTED_DT','REV_REPT_PREP_NAME','REVISED_DT','CORRECTED_DT','LICENSE_NO','LICENSE_EFF_DATE','LICENSE_EXP_DATE','LICENSE_STATUS','FACILITY_LEVEL','ASSEMBLY_DIST','SENATE_DIST','CONGRESS_DIST','CENS_TRACT','MED_SVC_STUDY_AREA','LA_COUNTY_SVC_PLAN_AREA','COUNTY'], axis=1)
# ,'RURAL_HEALTH_CLINIC']

# Format Categorical Data
df.TEACH_HOSP = df.TEACH_HOSP != "No"
df.TEACH_RURAL = df.TEACH_RURAL != "No"
df.TRAUMA_CTR = df.TRAUMA_CTR != 0
df.HEALTH_SVC_AREA = df.HEALTH_SVC_AREA.str[:2]
df.LIC_CAT = df.LIC_CAT == "General Acute Care Hospital"
df.LICEE_TOC = df.LICEE_TOC == "Non-Profit Corporation (including church-related)"
df.PRIN_SERVICE_TYPE = df.PRIN_SERVICE_TYPE == "General Medical / Surgical"
df.FAC_ACQUIRE_EQUIP_OVER_500K = df.FAC_ACQUIRE_EQUIP_OVER_500K != "No"
df.OFFER_ALTERNATE_BIRTH_PROG = df.OFFER_ALTERNATE_BIRTH_PROG != "No"
df.LIC_CARDIOLOGY_CARDIOVASCULAR_SURG_SERVICES = df.LIC_CARDIOLOGY_CARDIOVASCULAR_SURG_SERVICES != 0
df.OFFER_AMBULATORY_SURG_PROG = df.OFFER_AMBULATORY_SURG_PROG != "No"
df.LIC_ED_LEV_END = df.LIC_ED_LEV_END != 0
df.EMSA_TRAUMA_DESIGNATION_PEDIATRIC = df.EMSA_TRAUMA_DESIGNATION_PEDIATRIC != 0
df.EMER_DEPT_AMBULANCE_DIVERSION_HOURS = df.EMER_DEPT_AMBULANCE_DIVERSION_HOURS != "No"
df.LIC_ED_LEV_BEGIN = df.LIC_ED_LEV_BEGIN != 0
df.EMSA_TRAUMA_DESIGNATION = df.EMSA_TRAUMA_DESIGNATION != 0
df.OUTPATIENT_PALLIATIVE_CARE_SERV_OFFERED = df.OUTPATIENT_PALLIATIVE_CARE_SERV_OFFERED != 0
df.INPATIENT_PALLIATIVE_CARE_PROG_OFFERED = df.INPATIENT_PALLIATIVE_CARE_PROG_OFFERED != 0
df.INPATIENT_HOSPICE_PROG_OFFERED = df.INPATIENT_HOSPICE_PROG_OFFERED != 0
df.SHORT_DOYLE_SERVICES_OFFERED = df.SHORT_DOYLE_SERVICES_OFFERED != 0


# df.PRIMARY_NON_ENGLISH_LANG = df.PRIMARY_NON_ENGLISH_LANG == "Spanish"
df.replace(True, 1, inplace=True)

# Drop final coloumns
df.drop(df.iloc[:, 267:323], inplace=True, axis=1)

# Fill in all blanks with 0 (for deep learning)
df = df.fillna(0)

df.head()

,FAC_ZIP,FAC_PAR_CORP_ZIP,TRAUMA_CTR,TEACH_HOSP,TEACH_RURAL,LONGITUDE,LATITUDE,HEALTH_SVC_AREA,LIC_CAT,LICEE_TOC,...,CARD_CATH_PED_IP_THER_VST,CARD_CATH_PED_OP_THER_VST,CARDIAC_CATHETERIZATION_ADULT_INPAT_THERAPEUTIC_VISITS,CARDIAC_CATHETERIZATION_ADULT_OUTPAT_THERAPEUTIC_VISITS,CARDIAC_CATHETERIZATION_THERAPEUTIC_VISITS_TOT,INPATIENT_AVG_PER_SURGERY,OUTPATIENT_AVG_PER_SURGERY,FAC_ACQUIRE_EQUIP_OVER_500K,Target,Label
0,94501.0,94602,1.0,0.0,0.0,-122.253991,37.76266,05,1.0,0.0,...,0,0,0,0,0,105.0,75.2,1.0,0.059835,1.0
1,94705.0,95833,1.0,0.0,0.0,-122.257430,37.85645,05,1.0,1.0,...,0,0,0,0,0,127.6,75.8,1.0,0.012884,0.0
2,94609.0,0,1.0,0.0,0.0,-122.267470,37.83722,05,1.0,1.0,...,22,35,2,1,60,137.4,70.8,0.0,0.000000,0.0
3,94578.0,94602,1.0,0.0,0.0,-122.118190,37.70648,05,1.0,0.0,...,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.0
4,94704.0,95833,1.0,0.0,0.0,-122.269840,37.86373,05,1.0,1.0,...,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.0


In [34]:
# Retrive all possible inputs
features = df.iloc[:, 0:267]
features

,FAC_ZIP,FAC_PAR_CORP_ZIP,TRAUMA_CTR,TEACH_HOSP,TEACH_RURAL,LONGITUDE,LATITUDE,HEALTH_SVC_AREA,LIC_CAT,LICEE_TOC,...,CARDIOVASCULAR_SURG_OPER_ADULT_BYPASS_NOT_USED,CARDIOVASCULAR_SURG_OPER_BYPASS_NOT_USED_TOT,CARD_CATH_PED_IP_THER_VST,CARD_CATH_PED_OP_THER_VST,CARDIAC_CATHETERIZATION_ADULT_INPAT_THERAPEUTIC_VISITS,CARDIAC_CATHETERIZATION_ADULT_OUTPAT_THERAPEUTIC_VISITS,CARDIAC_CATHETERIZATION_THERAPEUTIC_VISITS_TOT,INPATIENT_AVG_PER_SURGERY,OUTPATIENT_AVG_PER_SURGERY,FAC_ACQUIRE_EQUIP_OVER_500K
0,94501.0,94602,1.0,0.0,0.0,-122.253991,37.762660,05,1.0,0.0,...,0,0,0,0,0,0,0,105.0,75.2,1.0
1,94705.0,95833,1.0,0.0,0.0,-122.257430,37.856450,05,1.0,1.0,...,0,0,0,0,0,0,0,127.6,75.8,1.0
2,94609.0,0,1.0,0.0,0.0,-122.267470,37.837220,05,1.0,1.0,...,3,34,22,35,2,1,60,137.4,70.8,0.0
3,94578.0,94602,1.0,0.0,0.0,-122.118190,37.706480,05,1.0,0.0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
4,94704.0,95833,1.0,0.0,0.0,-122.269840,37.863730,05,1.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,95901.0,95991,1.0,0.0,0.0,-121.593602,39.138805,02,1.0,1.0,...,89,89,0,0,369,358,727,145.5,84.8,0.0
474,93703.0,93611,1.0,0.0,0.0,-119.779521,36.778140,09,1.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
475,93940.0,93940,1.0,0.0,0.0,-121.892428,36.580786,08,1.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
476,95023.0,95023,1.0,0.0,0.0,-121.386700,36.835140,08,1.0,0.0,...,0,0,0,0,0,0,0,0.0,0.0,0.0


## Data Pre-Processing

In [35]:
# Define X and y
X = features
y = df["Label"]
print(X.shape, y.shape)

(478, 267) (478,)


In [36]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=101)

In [37]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [38]:
# # Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [39]:
# # Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create a Deep Learning Model

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [41]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=267))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [42]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 100)               26800     
_________________________________________________________________
dense_9 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 202       
Total params: 47,202
Trainable params: 47,202
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=3
)

Epoch 1/1000
Epoch 2/1000
Epoch 3/1000
Epoch 4/1000
Epoch 5/1000
Epoch 6/1000
Epoch 7/1000
Epoch 8/1000
Epoch 9/1000
Epoch 10/1000
Epoch 11/1000
Epoch 12/1000
Epoch 13/1000
Epoch 14/1000
Epoch 15/1000
Epoch 16/1000
Epoch 17/1000
Epoch 18/1000
Epoch 19/1000
Epoch 20/1000
Epoch 21/1000
Epoch 22/1000
Epoch 23/1000
Epoch 24/1000
Epoch 25/1000
Epoch 26/1000
Epoch 27/1000
Epoch 28/1000
Epoch 29/1000
Epoch 30/1000
Epoch 31/1000
Epoch 32/1000
Epoch 33/1000
Epoch 34/1000
Epoch 35/1000
Epoch 36/1000
Epoch 37/1000
Epoch 38/1000
Epoch 39/1000
Epoch 40/1000
Epoch 41/1000
Epoch 42/1000
Epoch 43/1000
Epoch 44/1000
Epoch 45/1000
Epoch 46/1000
Epoch 47/1000
Epoch 48/1000
Epoch 49/1000
Epoch 50/1000
Epoch 51/1000
Epoch 52/1000
Epoch 53/1000
Epoch 54/1000
Epoch 55/1000
Epoch 56/1000
Epoch 57/1000
Epoch 58/1000
Epoch 59/1000
Epoch 60/1000
Epoch 61/1000
Epoch 62/1000
Epoch 63/1000
Epoch 64/1000
Epoch 65/1000
Epoch 66/1000
Epoch 67/1000
Epoch 68/1000
Epoch 69/1000
Epoch 70/1000
Epoch 71/1000
Epoch 72/1000
E

Epoch 567/1000
Epoch 568/1000
Epoch 569/1000
Epoch 570/1000
Epoch 571/1000
Epoch 572/1000
Epoch 573/1000
Epoch 574/1000
Epoch 575/1000
Epoch 576/1000
Epoch 577/1000
Epoch 578/1000
Epoch 579/1000
Epoch 580/1000
Epoch 581/1000
Epoch 582/1000
Epoch 583/1000
Epoch 584/1000
Epoch 585/1000
Epoch 586/1000
Epoch 587/1000
Epoch 588/1000
Epoch 589/1000
Epoch 590/1000
Epoch 591/1000
Epoch 592/1000
Epoch 593/1000
Epoch 594/1000
Epoch 595/1000
Epoch 596/1000
Epoch 597/1000
Epoch 598/1000
Epoch 599/1000
Epoch 600/1000
Epoch 601/1000
Epoch 602/1000
Epoch 603/1000
Epoch 604/1000
Epoch 605/1000
Epoch 606/1000
Epoch 607/1000
Epoch 608/1000
Epoch 609/1000
Epoch 610/1000
Epoch 611/1000
Epoch 612/1000
Epoch 613/1000
Epoch 614/1000
Epoch 615/1000
Epoch 616/1000
Epoch 617/1000
Epoch 618/1000
Epoch 619/1000
Epoch 620/1000
Epoch 621/1000
Epoch 622/1000
Epoch 623/1000
Epoch 624/1000
Epoch 625/1000
Epoch 626/1000
Epoch 627/1000
Epoch 628/1000
Epoch 629/1000
Epoch 630/1000
Epoch 631/1000
Epoch 632/1000
Epoch 633/

## Quantify our Trained Model

In [45]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=3)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 3.1508775934388367, Accuracy: 0.7911392450332642


In [46]:
model.get_weights()

[array([[-0.10151322,  0.1123227 ,  0.06980763, ..., -0.03421363,
         -0.07464585, -0.00275001],
        [-0.07420017, -0.08563564, -0.22085555, ..., -0.11962539,
          0.0689491 , -0.03575121],
        [-0.1258318 , -0.07177928,  0.00767173, ...,  0.07086626,
          0.0949591 , -0.08170063],
        ...,
        [-0.07212465,  0.0413024 ,  0.01210163, ...,  0.07311409,
          0.08067045, -0.0305264 ],
        [ 0.08712982,  0.14333744, -0.00228881, ...,  0.05973327,
          0.16796568, -0.03634572],
        [ 0.06588777, -0.10590834,  0.16976215, ...,  0.01281627,
         -0.02098437,  0.05276867]], dtype=float32),
 array([ 0.01515077,  0.0001242 ,  0.00956865,  0.02596458,  0.01671748,
         0.03775145,  0.01462833,  0.00218778,  0.03498785,  0.0216798 ,
         0.00634036,  0.00819093, -0.00239146,  0.00810429,  0.0151864 ,
         0.02515629,  0.00704255,  0.02957255,  0.02429142,  0.0308438 ,
         0.01917454, -0.01204229,  0.01168754, -0.00019167, -0.002

## Make Predictions

In [47]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [48]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: [0. 0. 1. 1. 0.]
Actual Labels: [1.0, 0.0, 0.0, 1.0, 0.0]


In [55]:
predictions = model.predict_classes(X_test_scaled)
labels = label_encoder.inverse_transform(predictions)
# list(labels)

In [61]:
# list(y_test)
print(model.get_weights())

[array([[-0.10151322,  0.1123227 ,  0.06980763, ..., -0.03421363,
        -0.07464585, -0.00275001],
       [-0.07420017, -0.08563564, -0.22085555, ..., -0.11962539,
         0.0689491 , -0.03575121],
       [-0.1258318 , -0.07177928,  0.00767173, ...,  0.07086626,
         0.0949591 , -0.08170063],
       ...,
       [-0.07212465,  0.0413024 ,  0.01210163, ...,  0.07311409,
         0.08067045, -0.0305264 ],
       [ 0.08712982,  0.14333744, -0.00228881, ...,  0.05973327,
         0.16796568, -0.03634572],
       [ 0.06588777, -0.10590834,  0.16976215, ...,  0.01281627,
        -0.02098437,  0.05276867]], dtype=float32), array([ 0.01515077,  0.0001242 ,  0.00956865,  0.02596458,  0.01671748,
        0.03775145,  0.01462833,  0.00218778,  0.03498785,  0.0216798 ,
        0.00634036,  0.00819093, -0.00239146,  0.00810429,  0.0151864 ,
        0.02515629,  0.00704255,  0.02957255,  0.02429142,  0.0308438 ,
        0.01917454, -0.01204229,  0.01168754, -0.00019167, -0.00266363,
        0.

In [51]:
# Create the SVC Model
from sklearn.svm import SVC 
model2 = SVC(kernel='linear')
model2.fit(X_train_scaled, y_train)

# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model2, param_grid, verbose=3)

In [52]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.781, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.875, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.875, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.719, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.719, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.781, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ...........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



[CV] C=1, gamma=0.005 ................................................
[CV] .................... C=1, gamma=0.005, score=0.781, total=   0.0s
[CV] C=1, gamma=0.005 ................................................
[CV] .................... C=1, gamma=0.005, score=0.875, total=   0.0s
[CV] C=1, gamma=0.005 ................................................
[CV] .................... C=1, gamma=0.005, score=0.875, total=   0.0s
[CV] C=1, gamma=0.005 ................................................
[CV] .................... C=1, gamma=0.005, score=0.719, total=   0.0s
[CV] C=1, gamma=0.005 ................................................
[CV] .................... C=1, gamma=0.005, score=0.719, total=   0.0s
[CV] C=5, gamma=0.0001 ...............................................
[CV] ................... C=5, gamma=0.0001, score=0.766, total=   0.0s
[CV] C=5, gamma=0.0001 ...............................................
[CV] ................... C=5, gamma=0.0001, score=0.859, total=   0.0s
[CV] 

[CV] ................... C=50, gamma=0.001, score=0.828, total=   0.0s
[CV] C=50, gamma=0.001 ...............................................
[CV] ................... C=50, gamma=0.001, score=0.750, total=   0.0s
[CV] C=50, gamma=0.001 ...............................................
[CV] ................... C=50, gamma=0.001, score=0.781, total=   0.0s
[CV] C=50, gamma=0.005 ...............................................
[CV] ................... C=50, gamma=0.005, score=0.719, total=   0.0s
[CV] C=50, gamma=0.005 ...............................................
[CV] ................... C=50, gamma=0.005, score=0.797, total=   0.0s
[CV] C=50, gamma=0.005 ...............................................
[CV] ................... C=50, gamma=0.005, score=0.828, total=   0.0s
[CV] C=50, gamma=0.005 ...............................................
[CV] ................... C=50, gamma=0.005, score=0.750, total=   0.0s
[CV] C=50, gamma=0.005 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    1.2s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [53]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'gamma': 0.0001}
0.815625
